이번 챕터에선 문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용합니다. 그리고 단어 단위를 사용함에 따라서 파이토치에서 제공하는 임베딩 층을 사용하겠습니다.

## **1. 훈련 데이터 전처리하기**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

우리가 만들 RNN은 "Repeat is the best medicine for"을 입력받으면 "is the best medicine for memory"를 출력하는 RNN입니다. 위의 임의의 문장으로부터 단어장(vocabulary)을 만듭니다.

In [3]:
vocab = list(set(sentence))
vocab

['best', 'the', 'medicine', 'is', 'memory', 'Repeat', 'for']

이제 단어장의 단어에 고유한 정수 인덱스를 부여합니다. 그리고 그와 동시에 모르는 단어를 의미하는 UNK 토큰도 추가하겠습니다.

In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)} # 단어에 고유한 정수 부여
word2index['<unk>'] = 0

In [5]:
word2index

{'best': 1,
 'the': 2,
 'medicine': 3,
 'is': 4,
 'memory': 5,
 'Repeat': 6,
 'for': 7,
 '<unk>': 0}

이제 word2index가 우리가 사용할 최종 단어장인 셈입니다. word2index에 단어를 입력하면 맵핑되는 정수를 리턴합니다.

In [6]:
word2index['memory']

5

단어 'memory'와 맵핑되는 정수는 5입니다. 예측 단계에서 예측한 문장을 확인하기 위해 idx2word도 만듭니다.

In [7]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
index2word

{1: 'best',
 2: 'the',
 3: 'medicine',
 4: 'is',
 5: 'memory',
 6: 'Repeat',
 7: 'for',
 0: '<unk>'}

idx2word는 정수로부터 단어를 리턴하는 역할을 합니다. 정수 5를 넣어보겠습니다.

In [8]:
index2word[5]

'memory'

정수 5와 맵핑되는 단어는 memory인 것을 확인할 수 있습니다. 

이제 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수를 만들겠습니다.

In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입략 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

만들어진 함수로부터 입력 데이터와 레이블 데이터를 얻습니다.

In [10]:
X, Y = build_data(sentence, word2index)

In [11]:
print(X) # Repeat is the best medicine for을 의미
print(Y) # is the best medicine for memory를 의미

tensor([[6, 4, 2, 1, 3, 7]])
tensor([[4, 2, 1, 3, 7, 5]])


## **2. 모델 구현하기**

이제 모델을 설계합니다. 이전 모델들과 달라진 점은 임베딩 층을 추가했다는 점입니다. 파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현합니다. 임베딩 층은 크게 두 가지 인자를 받는데 첫 번째 인자는 단어장의 크기이며, 두번째 인자는 임베딩 벡터의 차원입니다.

In [12]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size) # 워드 임베딩
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first) # 입력 차원, 은닉 상태의 크기 정의
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함
    
    def forward(self, x):
        # 1. 임베딩 층
        # 크기 변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        
        # 2. RNN 층
        # 크기 변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        
        # 3. 최종 출력층
        # 크기 변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)'
        output = self.linear(output)
        
        # 4. view를 통해서 배치 차원 제거
        # 크기 변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기 * 시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [13]:
# 하이퍼 파라미터
vocab_size = len(word2index) # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5 # 임베딩 된 차원의 크기 및 RNN층 입력 차원의 크기
hidden_size = 20 # RNN의 은닉층 크기

In [14]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩은 안 해도 됩니다.

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

모델에 입력을 넣어서 출력을 확인해보겠습니다.

In [15]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화된 상태입니다.
output = model(X)
print(output)

tensor([[ 0.1801,  0.0945,  0.1071,  0.2345,  0.2330,  0.0383,  0.1650,  0.1490],
        [-0.3891,  0.0908,  0.2698, -0.0534, -0.4489, -0.3066, -0.5078,  0.4477],
        [ 0.1671,  0.1418,  0.3733, -0.0107,  0.1064,  0.0799,  0.1558,  0.1736],
        [ 0.0982,  0.0098, -0.0145,  0.2408,  0.2369,  0.4046,  0.1828,  0.2845],
        [-0.2013,  0.0893,  0.1455,  0.0742, -0.1523,  0.2154, -0.1256,  0.0819],
        [-0.1275,  0.1054,  0.2911, -0.0026, -0.3868, -0.2000, -0.3177, -0.0066]],
       grad_fn=<ViewBackward0>)


모델이 어떤 예측값을 내놓기는 하지만 현재 가중치는 랜덤 초기화되어 있어 의미있는 예측값은 아닙니다. 예측값의 크기를 확인해보겠습니다.

In [16]:
print(output.shape)

torch.Size([6, 8])


예측값의 크기는 (6, 8)입니다. 이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당됩니다. 모델은 훈련시키기 전에 예측을 제대로 하고 있는지, 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 decode 함수를 만듭니다.

In [17]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [18]:
# 훈련 시작
for step in range(201):
    optimizer.zero_grad() # 경사 초기화
    output = model(X) # 순방향 전파
    loss = loss_function(output, Y.view(-1)) # 손실값 계산
    loss.backward() # 역방향 전파
    optimizer.step() # 매개변수 업데이트
    
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0195 
Repeat medicine for the memory memory the

[41/201] 1.3851 
Repeat is the best medicine for memory

[81/201] 0.7934 
Repeat is the best medicine for memory

[121/201] 0.4442 
Repeat is the best medicine for memory

[161/201] 0.2601 
Repeat is the best medicine for memory

[201/201] 0.1649 
Repeat is the best medicine for memory

